# Modeling and Forecasting Notebook

This notebook is a continuation of the previous **EDA (Exploratory Data Analysis) notebook**. Having explored and cleaned the data in the EDA phase, this notebook focuses on:

- **Preparing the data** for modeling (including filling missing dates to ensure continuous weekly data)
- **Applying feature engineering** techniques such as lag features, rolling averages, and capping to improve predictive power
- **Training XGBoost models** for each `SerialNum`
- **Evaluating model performance** using accuracy metrics on a 3-month hold-out validation set
- **Forecasting weekly sales** for the next 3 months (September, October, and November 2024) as per the assignment requirement

Feature engineering steps are modular, allowing toggling on/off of:
- Lag features (with adjustable lag window)
- Rolling means (with configurable window sizes)
- Capping/extreme value control

This design allows us to experiment and evaluate the impact of each technique on model accuracy.

The final output of this notebook includes:
 - Monthly and overall validation accuracy  
 - Forecasted weekly sales for Sep–Nov 2024  

All steps are designed to ensure **consistent and explainable demand forecasting**, aligned with business requirements.




Firstly, we import the essential libraries required for the analysis and modeling:

In [71]:
# Import libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import joblib

import warnings
warnings.filterwarnings('ignore')




Next, we first load our cleaned dataset from the CSV file into a pandas DataFrame so that we can begin analysis and modeling. We then convert the weekend_date column into a proper datetime format, explicitly specifying dayfirst=True because our data uses day-first dates. After ensuring that our dates are in datetime format, we sort the data by SerailNum and weekend_date to maintain the correct chronological order for each time series, which is essential for creating lag features and ensuring the model respects time dependencies. Finally, we reset the index to keep the DataFrame tidy and print a quick preview to confirm the data has loaded and sorted correctly.

In [72]:
# Load data
data_path = 'cleaned_Assessment.csv'
df = pd.read_csv(data_path)
df['weekend_date'] = pd.to_datetime(df['weekend_date'], dayfirst=True)

# Sort for time order
df = df.sort_values(['SerailNum', 'weekend_date']).reset_index(drop=True)
print(df.head())


  weekend_date   channel brand category sub_category  SerailNum  quantity
0   2022-06-04  Channel1    B1     Cat2     Sub-Cat2          1        56
1   2022-06-11  Channel1    B1     Cat2     Sub-Cat2          1       122
2   2022-06-18  Channel1    B1     Cat2     Sub-Cat2          1       102
3   2022-06-25  Channel1    B1     Cat2     Sub-Cat2          1       128
4   2022-07-02  Channel1    B1     Cat2     Sub-Cat2          1        97


Now, we ensure that our time series data has no missing weekly intervals for each unique SerailNum. Forecasting models typically expect a complete, regular time grid. To achieve this, we first extract the unique SerailNum values and process them one at a time. For each serial, we determine its start and end dates and generate a continuous sequence of Saturdays between them using pd.date_range(). We then merge this full calendar with the existing data so that any missing weeks become visible as rows with nulls. To preserve metadata consistency (such as channel, brand, category, and sub_category), we forward-fill these columns—assuming these attributes remain constant over time. Finally, any missing quantity values (which represent missing sales for those weeks) are filled with zeros, indicating no sales in those weeks. We repeat this for all serials, concatenate them into a single DataFrame, and print the head to verify that our time grid is complete and the data is clean.

In [73]:
# Fill missing weekly dates
filled_dfs = []
serials = df['SerailNum'].unique()

for serial in serials:
    df_serial = df[df['SerailNum'] == serial].copy()
    start_date = df_serial['weekend_date'].min()
    end_date = df_serial['weekend_date'].max()
    full_dates = pd.DataFrame({
        'weekend_date': pd.date_range(start=start_date, end=end_date, freq='W-SAT')
    })
    merged = full_dates.merge(df_serial, on='weekend_date', how='left')
    for col in ['channel', 'brand', 'category', 'sub_category', 'SerailNum']:
        merged[col] = merged[col].fillna(method='ffill')
    merged['quantity'] = merged['quantity'].fillna(0)
    filled_dfs.append(merged)

df_filled = pd.concat(filled_dfs).reset_index(drop=True)
print(df_filled.head())


  weekend_date   channel brand category sub_category  SerailNum  quantity
0   2022-06-04  Channel1    B1     Cat2     Sub-Cat2        1.0      56.0
1   2022-06-11  Channel1    B1     Cat2     Sub-Cat2        1.0     122.0
2   2022-06-18  Channel1    B1     Cat2     Sub-Cat2        1.0     102.0
3   2022-06-25  Channel1    B1     Cat2     Sub-Cat2        1.0     128.0
4   2022-07-02  Channel1    B1     Cat2     Sub-Cat2        1.0      97.0


Next, we define switches to control feature engineering steps.

- USE_LAG and LAG_N set whether to add lag features and how many weeks.

- USE_ROLLING and ROLLING_WINDOWS set rolling average features.

- USE_CAPPING and CAPPING_VALUE control outlier capping.

By changing these flags, we can easily test different modeling strategies without rewriting code.

In [74]:
# Feature Engineering Switches
USE_LAG = True
LAG_N = 4

USE_ROLLING = True
ROLLING_WINDOWS = [3, 6]

USE_CAPPING = True
CAPPING_VALUE = 0.99 # e.g., 99th percentile

print(f"Switches - USE_LAG={USE_LAG}, USE_ROLLING={USE_ROLLING}, USE_CAPPING={USE_CAPPING}")

Switches - USE_LAG=True, USE_ROLLING=True, USE_CAPPING=True


We now engineer features to improve model learning:

- Lag Features: Add past values of quantity to capture temporal dependencies.

- Rolling Features: Compute smoothed averages over time windows.

- Capping: Limit extreme values to reduce outlier impact.

Each transformation is controlled by switches so you can easily turn them on or off. The result is a flexible design to experiment with different feature combinations.

In [75]:
# Feature Engineering
df_features = df_filled.copy()
df_features = df_features.sort_values(['SerailNum', 'weekend_date']).reset_index(drop=True)

if USE_LAG:
    for lag in range(1, LAG_N + 1):
        df_features[f'quantity_lag_{lag}'] = df_features.groupby('SerailNum')['quantity'].shift(lag)

if USE_ROLLING:
    for window in ROLLING_WINDOWS:
        df_features[f'quantity_roll{window}'] = df_features.groupby('SerailNum')['quantity'].transform(lambda x: x.rolling(window).mean())

if USE_CAPPING:
    cap_value = df_features['quantity'].quantile(CAPPING_VALUE) if CAPPING_VALUE else df_features['quantity'].max()
    df_features['quantity_capped'] = df_features['quantity'].clip(upper=cap_value)
    target_col = 'quantity_capped'
else:
    target_col = 'quantity'

print(df_features.head(10))


  weekend_date   channel brand category sub_category  SerailNum  quantity  \
0   2022-06-04  Channel1    B1     Cat2     Sub-Cat2        1.0      56.0   
1   2022-06-11  Channel1    B1     Cat2     Sub-Cat2        1.0     122.0   
2   2022-06-18  Channel1    B1     Cat2     Sub-Cat2        1.0     102.0   
3   2022-06-25  Channel1    B1     Cat2     Sub-Cat2        1.0     128.0   
4   2022-07-02  Channel1    B1     Cat2     Sub-Cat2        1.0      97.0   
5   2022-07-09  Channel1    B1     Cat2     Sub-Cat2        1.0     114.0   
6   2022-07-16  Channel1    B1     Cat2     Sub-Cat2        1.0      84.0   
7   2022-07-23  Channel1    B1     Cat2     Sub-Cat2        1.0     102.0   
8   2022-07-30  Channel1    B1     Cat2     Sub-Cat2        1.0     115.0   
9   2022-08-06  Channel1    B1     Cat2     Sub-Cat2        1.0      91.0   

   quantity_lag_1  quantity_lag_2  quantity_lag_3  quantity_lag_4  \
0             NaN             NaN             NaN             NaN   
1            5

Next task i to split the data into training (older dates) and validation (last 3 months) sets. This simulates real forecasting by ensuring the model is trained on past data and tested on future periods without data leakage.

In [76]:
# Train/Validation Split
train_cutoff_date = df_features['weekend_date'].max() - pd.DateOffset(months=3)
train_df = df_features[df_features['weekend_date'] <= train_cutoff_date]
val_df = df_features[df_features['weekend_date'] > train_cutoff_date]

print(f"Training up to: {train_cutoff_date}")
print(f"Train rows: {len(train_df)}, Val rows: {len(val_df)}")


Training up to: 2024-05-31 00:00:00
Train rows: 528, Val rows: 75


Now, we train a separate XGBoost regression model for each SerialNum using the engineered features—lags and rolling averages. For each SerialNum, we filter its training data, drop rows with missing feature values (which can occur due to lags/rolling), and train a model to predict the quantity. The trained models are saved using joblib for later use in validation or forecasting. This approach allows each product to be modeled individually, capturing its unique trends and patterns.



In [77]:
# Train XGBoost Models by SerialNum
feature_cols = [col for col in df_features.columns if 'lag' in col or 'roll' in col]
print(f"Using features: {feature_cols}")

serials = df_features['SerailNum'].unique()

for serial in serials:
    train_serial = train_df[train_df['SerailNum'] == serial].dropna(subset=feature_cols)
    if train_serial.empty:
        print(f" No training rows for SerialNum {serial}")
        continue
    X_train = train_serial[feature_cols]
    y_train = train_serial[target_col]
    model = xgb.XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    joblib.dump(model, f"xgb_model_serial_{int(serial)}.joblib")
    print(f"Trained model for SerialNum {serial}")


Using features: ['quantity_lag_1', 'quantity_lag_2', 'quantity_lag_3', 'quantity_lag_4', 'quantity_roll3', 'quantity_roll6']
Trained model for SerialNum 1.0
Trained model for SerialNum 2.0
Trained model for SerialNum 3.0
Trained model for SerialNum 4.0
Trained model for SerialNum 5.0


Next, we evaluate our trained models on the validation set. For each SerialNum, we load its trained XGBoost model and use it to predict the quantity on its validation data. We ensure that only rows with valid (non-NaN) feature values are used. The predictions (y_hat) are stored alongside actual values to compute errors. We also extract the month from the date to enable later monthly aggregation. Finally, we calculate the absolute error between predicted and actual quantity, preparing the data for accuracy analysis.

In [78]:
# Validation Predictions
all_val_preds = []

for serial in serials:
    model_path = f"xgb_model_serial_{int(serial)}.joblib"
    try:
        model = joblib.load(model_path)
    except:
        continue

    val_serial = val_df[val_df['SerailNum'] == serial].copy()
    val_serial = val_serial.dropna(subset=feature_cols)
    if val_serial.empty:
        continue

    X_val = val_serial[feature_cols]
    val_serial['y_hat'] = model.predict(X_val)
    all_val_preds.append(val_serial)

val_predictions = pd.concat(all_val_preds).reset_index(drop=True)
val_predictions['month'] = val_predictions['weekend_date'].dt.month
val_predictions['error'] = abs(val_predictions['y_hat'] - val_predictions[target_col])


Toevaluate model performance using intuitive accuracy metrics:

- First, we group predictions by SerialNum and month to see how well each model performs for each product/series over time. We sum the absolute error and actual quantity to get an aggregated monthly accuracy per SerialNum.

- Then, we compute an overall accuracy across all data by comparing total prediction error to total actual quantity.

- Finally, we calculate accuracy per month across all products to identify seasonal or time-based variation in performance.

This helps us understand and report model accuracy both overall and in detail, as required in the assignment.



In [80]:
# Accuracy by SerialNum and Month
monthly_results = val_predictions.groupby(['SerailNum', 'month']).agg({
    'error': 'sum',
    target_col: 'sum'
}).reset_index()
monthly_results['accuracy'] = 1 - (monthly_results['error'] / monthly_results[target_col])
print(" Monthly Accuracy by SerialNum")
print(monthly_results)

# Overall Accuracy
overall_error = val_predictions['error'].sum()
overall_quantity = val_predictions[target_col].sum()
overall_accuracy = 1 - (overall_error / overall_quantity)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Monthly Accuracy
month_level = val_predictions.groupby('month').agg({
    'error': 'sum',
    target_col: 'sum'
}).reset_index()
month_level['accuracy'] = 1 - (month_level['error'] / month_level[target_col])
print("Monthly Accuracy Overall")
print(month_level)


 Monthly Accuracy by SerialNum
    SerailNum  month        error  quantity_capped   accuracy
0         1.0      6    53.106384           148.00   0.641173
1         1.0      7    30.926054           165.00   0.812569
2         1.0      8    54.512875           297.00   0.816455
3         2.0      6   655.100006          3488.00   0.812185
4         2.0      7   134.122696           742.00   0.819242
5         2.0      8   123.327438           903.00   0.863425
6         3.0      6   794.288002          1269.00   0.374084
7         3.0      7   426.382507           170.00  -1.508132
8         3.0      8   505.143089           921.00   0.451528
9         4.0      6  6317.679749         16011.00   0.605416
10        4.0      7  4793.930144         14485.72   0.669058
11        4.0      8  1444.043274          8235.00   0.824646
12        5.0      6  1192.965431            15.00 -78.531029
Overall Accuracy: 0.6473
Monthly Accuracy Overall
   month        error  quantity_capped  accuracy
0 

##Interpretation of Results

After running the model with the chosen feature engineering settings (lags, rolling averages, capping), we evaluated its performance on the validation set:

Monthly Accuracy by SerialNum: Shows how well the model predicts for each product (SerialNum) in each month. We see good accuracy for many products (often > 0.8), though some SerialNums have poor or even negative accuracy, suggesting they may need further investigation or may be harder to predict.

Overall Accuracy: ~64.7%, indicating that in total, the model explains a good portion of the quantity variation over the entire validation period.

Monthly Accuracy Overall: Breaks down accuracy across all products for June, July, and August. Accuracy improves over the months (from ~56.9% in June to ~79.5% in August), possibly reflecting seasonality or data distribution shifts.

These results help us understand where the model is strong and where it needs improvement, supporting decisions about further tuning or feature engineering.

#Forecasting Sales for Sep–Nov
Finally, I generate predictions for all Saturdays in September, October, and November 2024 (as required in the question). I used the trained XGBoost models for each SerialNum to forecast sales for these 13 weeks.

The forecasting approach simulates each future week step by step. For each week:

Lag Features: Past predicted values are used to compute lag inputs.

Rolling Means: Rolling averages over prior predicted quantities are updated recursively.

Capping: The same target transformation is used as in training.

This recursive simulation ensures consistency with the training features, allowing the model to project future demand realistically. The final output is a table of forecasted quantities for each SerialNum and each Saturday in the target months.



In [81]:
# Forecasting for Sep-Nov
import pandas as pd

# Define forecast range specifically for Sep, Oct, Nov
forecast_start = pd.Timestamp('2024-09-01')
forecast_end = pd.Timestamp('2024-11-30')

# Generate all Saturdays in this range
forecast_dates = pd.date_range(
    start=forecast_start,
    end=forecast_end,
    freq='W-SAT'
)
print("Forecast Weeks:", forecast_dates)

all_forecasts = []

for serial in serials:
    print(f"\nGenerating forecast for SerialNum {serial}")

    # Start with this serial's existing data
    serial_history = df_features[df_features['SerailNum'] == serial].copy()
    serial_history = serial_history.sort_values('weekend_date')

    # Copy to simulate appending forecasts
    forecast_df = serial_history.copy()

    for date in forecast_dates:
        new_row = {
            'SerailNum': serial,
            'weekend_date': date
        }

        # Lag features
        if USE_LAG:
            for lag in range(1, LAG_N + 1):
                if len(forecast_df) >= lag:
                    new_row[f'quantity_lag_{lag}'] = forecast_df.iloc[-lag][target_col]
                else:
                    new_row[f'quantity_lag_{lag}'] = np.nan

        # Rolling features
        if USE_ROLLING:
            for window in ROLLING_WINDOWS:
                if len(forecast_df) >= window:
                    new_row[f'quantity_roll{window}'] = forecast_df[target_col].iloc[-window:].mean()
                else:
                    new_row[f'quantity_roll{window}'] = np.nan

        # Drop NA if lags/rollings insufficient
        feature_input = pd.DataFrame([new_row])
        feature_input = feature_input.dropna(subset=feature_cols)
        if feature_input.empty:
            print(f" Skipping forecast for {serial} on {date.date()} due to insufficient lags/rollings")
            continue

        # Load model
        try:
            model = joblib.load(f"xgb_model_serial_{int(serial)}.joblib")
        except:
            print(f" Model for SerialNum {serial} not found")
            continue

        # Predict
        new_row[target_col] = model.predict(feature_input[feature_cols])[0]
        forecast_df = pd.concat([forecast_df, pd.DataFrame([new_row])], ignore_index=True)

        all_forecasts.append({
            'SerailNum': serial,
            'weekend_date': date,
            'forecast_quantity': new_row[target_col]
        })

forecast_result = pd.DataFrame(all_forecasts)
print("\n Forecast Results Sample")
print(forecast_result.head())


Forecast Weeks: DatetimeIndex(['2024-09-07', '2024-09-14', '2024-09-21', '2024-09-28',
               '2024-10-05', '2024-10-12', '2024-10-19', '2024-10-26',
               '2024-11-02', '2024-11-09', '2024-11-16', '2024-11-23',
               '2024-11-30'],
              dtype='datetime64[ns]', freq='W-SAT')

Generating forecast for SerialNum 1.0

Generating forecast for SerialNum 2.0

Generating forecast for SerialNum 3.0

Generating forecast for SerialNum 4.0

Generating forecast for SerialNum 5.0

 Forecast Results Sample
   SerailNum weekend_date  forecast_quantity
0        1.0   2024-09-07          55.541859
1        1.0   2024-09-14          41.585884
2        1.0   2024-09-21          47.812176
3        1.0   2024-09-28          51.837296
4        1.0   2024-10-05          43.341766
